In [ ]:
from datetime import datetime, timedelta
from urllib.request import urlopen
import csv
import requests


data = []


def api_call(city, pollutant, day):
    day = str(day)
    day = day[:10]
    print("\n")
    print(city, pollutant, day)
    api_test = (
        "https://api.openaq.org/v2/measurements?city="
        + city
        + "&parameter="
        + pollutant
        + "&date_to="
        + day
        + "&unit=µg/m³&limit=10000&format=csv"
    )
    with requests.Session() as s:
        download = s.get(api_test)

        decoded_content = download.content.decode("utf-8")

        cr = csv.reader(decoded_content.splitlines(), delimiter=",")
        my_list = list(cr)
        print(len(my_list))
        for row in my_list:
            print(row)
    return my_list[1:]

cities = ['Kolkata']
# cities = ['Mumbai', 'Delhi', 'Kolkata', 'Chennai', 'Pune', 'Agra', 'Bengaluru', 'Jaipur', 'Hyderabad', 'Indore','Ahmedabad','Alwar','Amaravati','Aizawl','Belgaum','Chandigarh','Dehradun','Gangtok','Gurugram','Gwalior','Imphal','Jabalpur','Lucknow',"Kota","Ludhiana","Muzaffarnagar","Prayagraj","Shillong","Siliguri","Solapur","Tirupati","Ujjain","Vapi","Visakhapatnam"]
# cities = ['Mumbai', 'Delhi', 'Kolkata', 'Chennai', 'Pune', 'Agra', 'Bengaluru', 'Jaipur', 'Hyderabad', 'Indore','Ahmedabad'] #Mohsin
# cities = ['Alwar','Amaravati','Aizawl','Belgaum','Chandigarh','Dehradun','Gangtok','Gurugram','Gwalior','Imphal','Jabalpur'] #Hardik
# cities = ['Lucknow',"Kota","Ludhiana","Muzaffarnagar","Prayagraj","Shillong","Siliguri","Solapur","Tirupati","Ujjain","Vapi","Visakhapatnam"] #Jeet


start_date = datetime(2017, 1, 1)  # Year-Month-Date
# end_date = datetime(2018, 4, 1)  # perhaps date.now()
end_date = datetime.today()  # perhaps date.now()
day = datetime.today()
delta = end_date - start_date  # returns timedelta
print(delta.days)
pollutants = ["pm10", "pm25", "so2", "no2", "co", "o3"]
# pollutants = ['pm10']
# CO needs to be read in mg/m3. For now we are reading it in µg/m³, but later we will have to convert it to calculate AQI correctly.

# api = 'https://api.openaq.org/v1/measurements?city=Ahmedabad&parameter=pm25&date_from=2020-04-01&date_to=2020-04-10&format=json'

print("Start")
for city in cities:
    print("\nCity - " + city)
    data = []
    for pollutant in pollutants:
        print("Pollutant - " + pollutant)
        day = datetime.today()
        while(day >= start_date):
          delta = day - start_date
          api_data = api_call(city, pollutant, day)
          if len(api_data) <= 1:
              day = day - timedelta(days=1)
          else:
              year = int(api_data[-1][4][:4])
              month = int(api_data[-1][4][5:7])
              date = int(api_data[-1][4][8:10])
              day = datetime(year,month,date) - timedelta(days=1)
          for row in api_data:
              data.append(row)


    fields = [
        "locationId",
        "location",
        "city",
        "country",
        "utc",
        "local",
        "parameter",
        "value",
        "unit",
        "latitude",
        "longitude",
    ]
    with open(
        "datasets/AQIdata_"
        + city
        + "_new.csv",
        "w",
    ) as f:

        # using csv.writer method from CSV package
        write = csv.writer(f)

        write.writerow(fields)
        write.writerows(data)


print("Done")


Streaming output truncated to the last 5000 lines.
['716', 'Rabindra Bharati University, Kolkata - WBSPCB', 'Kolkata', 'IN', '2016-09-18T01:45:00+00:00', '2016-09-18T07:15:00+05:30', 'o3', '18.62', 'µg/m³', '22.627875', '88.3804']
['910', 'Victoria Memorial - WBSPCB', 'Kolkata', 'IN', '2016-09-18T01:30:00+00:00', '2016-09-18T07:00:00+05:30', 'o3', '7.29', 'µg/m³', '22.572646', '88.363895']
['910', 'Victoria Memorial - WBSPCB', 'Kolkata', 'IN', '2016-09-18T01:15:00+00:00', '2016-09-18T06:45:00+05:30', 'o3', '10.31', 'µg/m³', '22.572646', '88.363895']
['716', 'Rabindra Bharati University, Kolkata - WBSPCB', 'Kolkata', 'IN', '2016-09-18T01:15:00+00:00', '2016-09-18T06:45:00+05:30', 'o3', '12', 'µg/m³', '22.627875', '88.3804']
['716', 'Rabindra Bharati University, Kolkata - WBSPCB', 'Kolkata', 'IN', '2016-09-18T00:45:00+00:00', '2016-09-18T06:15:00+05:30', 'o3', '8.14', 'µg/m³', '22.627875', '88.3804']
['910', 'Victoria Memorial - WBSPCB', 'Kolkata', 'IN', '2016-09-18T00:45:00+00:00', '201

In [ ]:
import pandas as pd
# cities = [
#     "Kota",
#     "Ludhiana",
#     "Muzaffarnagar",
#     "Prayagraj",
#     "Shillong",
#     "Siliguri",
#     "Solapur",
#     "Tirupati",
#     "Ujjain",
#     "Vapi",
#     "Visakhapatnam",
# ]
path = "datasets/AQIdata_Mumbai_new.csv"
for city in cities:
  print(city)
  filename = "datasets/AQIdata_"+city+"_new.csv"
  df=pd.read_csv("datasets/AQIdata_"+city+"_new.csv",header=0)
  df.head()
  print("Before = ",len(df))
  df.drop_duplicates(subset=None, keep='first', inplace=True)
  print("Only Unique")
  print("After = ",len(df))

  df.to_csv('datasets/AQIdata_'+city+'_new.csv',index=False)
  print("Done\n")

Kolkata
Before =  1123053
Only Unique
After =  1123053
Done

